In [24]:
import pandas as pd
import os
import warnings

pd.set_option('display.max_rows', 50)

## carga de datos

se cargan los datos desde la carpeta `data`, la cual en nuestra arquitectura de nube vendria a ser nuestro data lake.

In [14]:
def load_data():
    warnings.filterwarnings("ignore")

    PATH_TO_DATA = "data"

    datos = []
    for file in os.listdir(PATH_TO_DATA):
        datos.append( pd.read_csv(os.path.join(PATH_TO_DATA, file), encoding="latin-1") )

    datos = pd.concat(datos, axis=0, ignore_index=True)

    return datos

## seleccion de columnas

**Los datos vienen inicialmente con las siguientes columnas**

- **BARRIO** : barrio de ocurrencia del incidente vial

- **CBML**: Codigo catastral que corresponde al codigo comuna, barrio, manzana, lote catastral  de un predio.

- **CLASE**: Clasificación del IPAT sobre la clase de accidente de transito: choque, atropello, volcamiento, caida de ocupante, incendio,  u otro (que no corresponde a las anteriores 5 clasificaciones, p. ej: sumersión)

- **COMUNA**: Comuna de ocurrencia del incidente vial

- **DIA**: Dia del mes de ocurrencia del incidente vial

- **DIA_NOMBRE**: Día en que tiene ocurrencia del incidente vial

- **DIRECCION**: Dirección donde ocurrió el incidente

- **DIRECCION_ENC**: Dirección encasillada que entrega el geocodificador

- **DISENO**: Sitio de la vía donde ocurrió el accidente: Cicloruta, Glorieta, Interseccion, Lote o Predio, Paso a Nivel, Paso Elevado, Paso Inferior, Pontón, Puente, Tramo de via, Tunel, Via peatonal

- **FECHA**: Fecha del incidente, proviene del IPAT - Informe Policial de incidentes de Tránsito 

- **GRAVEDAD**: Clasificación del IPAT - Informe Policial de Accidentes de Tránsito, sobre la gravedad del accidente, corresponde al resultado más grave presentado en el accidente. Daños materiales "Sólo daños", accidente con heridos "Herido", accidente con muertos "Muerto". No indica cantidad

- **HORA**: Hora del incidente, proviene del IPAT - Informe Policial de incidentes de Tránsito 

- **LATITUD**: Coordenada norte-sur para expresar la ubicación geográfica en el sistema de coordenadas WGS 84

- **LONGITUD**: Coordenada este-oeste para expresar la ubicación geográfica en el sistema de coordenadas WGS 84

- **MES**: Número de mes de ocurrencia del incidente vial

- **MES_NOMBRE**: Nombre del mes de ocurrencia del incidente vial

- **OBJECTID**: no esta en el diccionario de datos, sin embargo se puede suponer que es un identificador del registro

- **PERIODO**: Año de ocurrencia del incidente

- **RADICADO**: Consecutivo que asigna UNE, según el orden de llegada de los expedientes para su diligenciamiento

- **Shape**: no esta en el diccionario de datos, y no se sabe bien que representa

- **TIPO_GEOCOD**: Fuente de información con la cual se realizó la geocodificación

- **X_MAGNAMED**: Coordenada X en metros, en sistema de coordenadas MAGNA Medellin Local (MAGNA_Medellin_Antioquia_2010 WKID: 6257 Authority: EPSG)

- **Y_MAGNAMED**: Coordenada Y en metros, en sistema de coordenadas MAGNA Medellin Local (MAGNA_Medellin_Antioquia_2010 WKID: 6257 Authority: EPSG)


se seleccionan las columnas que son importantes para el analisis

In [36]:
def select_columns(datos):
    droped_cols = [
        "OBJECTID", # no es util para el analisis
        "Shape",    # no se sabe que es
        "RADICADO", # no es util para el analisis
    ]

    return datos.drop(columns=droped_cols)

## limpieza de los datos

se evalua que informacion requiere una limpieza

In [91]:
datos = load_data()
datos = select_columns(datos)

datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315087 entries, 0 to 315086
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   FECHA          315087 non-null  object 
 1   HORA           315087 non-null  object 
 2   DIA            315087 non-null  int64  
 3   PERIODO        315087 non-null  int64  
 4   CLASE          315081 non-null  object 
 5   DIRECCION      315087 non-null  object 
 6   DIRECCION_ENC  313785 non-null  object 
 7   CBML           292794 non-null  object 
 8   TIPO_GEOCOD    315087 non-null  object 
 9   GRAVEDAD       315087 non-null  object 
 10  BARRIO         291948 non-null  object 
 11  COMUNA         293830 non-null  object 
 12  DISENO         313803 non-null  object 
 13  DIA_NOMBRE     315087 non-null  object 
 14  MES            315087 non-null  int64  
 15  MES_NOMBRE     86394 non-null   object 
 16  X_MAGNAMED     315087 non-null  float64
 17  Y_MAGNAMED     315087 non-nul

#### se hicieron los siguientes hallazgos:

 - **FECHA**: no tiene el tipo de dato correcto, se convierte a tipo fecha

 - **HORA**: no tiene el tipo de dato correcto, se convierte a tipo hora

 - **CLASE**: contiene algunos valores nulos, se reemplazan estos valores con la palabra `No registra`

 - **DIRECCION_ENC**: contiene algunos valores nulos, se reemplazan estos valores con la palabra `No registra`

 - **CBML**: contiene algunos valores nulos, se reemplazan estos valores con la palabra `No registra`

 - **BARRIO**: contiene algunos valores nulos, se reemplazan estos valores con la palabra `No registra`

 - **COMUNA**: contiene algunos valores nulos, se reemplazan estos valores con la palabra `No registra`

 - **DISENO**: contiene algunos valores nulos, se reemplazan estos valores con la palabra `No registra`

 - **MES_NOMBRE**: contiene algunos valores nulos, se reemplazan estos valores segun su valor en la columna `MES`

In [100]:
def clean_data(datos):
    # eliminar datos duplicados
    datos.drop_duplicates(inplace=True)

    # convertir fechas a tipo de dato datetime
    datos["FECHA"] = pd.to_datetime(datos["FECHA"], format="%Y-%m-%d %H:%M:%S")

    # convertir hora a tipo de dato time
    datos["HORA"] = pd.to_datetime(datos["HORA"], format="mixed").dt.time

    # reemplazar valores nulos
    cols_na = ["CLASE", "DIRECCION_ENC", "CBML", "BARRIO", "COMUNA", "DISENO"]
    for col in cols_na:
        datos[col].fillna("No registra", inplace=True)

    meses = {1: "ENERO", 2: "FEBRERO", 3: "MARZO", 4: "ABRIL", 5: "MAYO", 6: "JUNIO", 7: "JULIO", 8: "AGOSTO", 9: "SEPTIEMBRE", 10: "OCTUBRE", 11: "NOVIEMBRE", 12: "DICIEMBRE"}
    datos["MES_NOMBRE"] = datos["MES"].apply(lambda x: meses[x])
    
    return datos

## datos limpios

In [102]:
def get_data():
    datos = load_data()
    datos = select_columns(datos)
    datos = clean_data(datos)

    return datos

In [103]:
datos = get_data()

datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 315087 entries, 0 to 315086
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   FECHA          315087 non-null  datetime64[ns]
 1   HORA           315087 non-null  object        
 2   DIA            315087 non-null  int64         
 3   PERIODO        315087 non-null  int64         
 4   CLASE          315087 non-null  object        
 5   DIRECCION      315087 non-null  object        
 6   DIRECCION_ENC  315087 non-null  object        
 7   CBML           315087 non-null  object        
 8   TIPO_GEOCOD    315087 non-null  object        
 9   GRAVEDAD       315087 non-null  object        
 10  BARRIO         315087 non-null  object        
 11  COMUNA         315087 non-null  object        
 12  DISENO         315087 non-null  object        
 13  DIA_NOMBRE     315087 non-null  object        
 14  MES            315087 non-null  int64         
 15  